# 환경 변수 로딩

In [20]:
from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = "wine-review2"
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

In [3]:
print(PINECONE_NAMESPACE)

wine-reviews-ns


# document loader

In [4]:
from langchain_community.document_loaders import CSVLoader
loader = CSVLoader('./wine_reviews/winemag-data-130k-v2.csv')
docs = loader.load()
for i, d in enumerate(docs[:3]):
    print(i)
    print(d)

0
page_content=': 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia' metadata={'source': './wine_reviews/winemag-data-130k-v2.csv', 'row': 0}
1
page_content=': 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Quinta dos Avidagos 2011 Avidagos Red 

# embedding 모델 객체 생성

In [17]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="llama-text-embed-v2",
    base_url="https://api.together.xyz/v1",
    api_key="OPENAI_API_KEY"
)

# Pinecone객체, index객체 생성

In [5]:
from pinecone import Pinecone, ServerlessSpec
# Pinecone 클라이언트를 초기화(객체생성)
pc = Pinecone(api_key=PINECONE_API_KEY)

In [6]:
# index 삭제
for idx in pc.list_indexes():
    print(idx.name)
    if idx.name == PINECONE_INDEX_NAME:
        pc.delete_index(idx.name)

lowbirth
wiki
wine-reviews
wine-review2


### pinecone 객체 생성, index 생성


In [9]:
index = pc.Index("wine-review2")

# split 객체 생성

In [10]:
# split하기
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 (예: 1000자씩 분할)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=100,
    # length_function=tiktoken_len,  # 토큰 기반 길이 측정    
    length_function=len,  # 문자수   
    separators=["\n\n", "\n", " ", ""]
    )

# 문서를 분할
chunks = text_splitter.split_documents(docs)

In [11]:
len(chunks)

129982

In [24]:
from pinecone import Pinecone
from uuid import uuid4

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("wine-review2")

In [29]:
from uuid import uuid4

BATCH_SIZE = 50

for start in range(0, len(chunks), BATCH_SIZE):
    end = start + BATCH_SIZE
    batch_docs = chunks[start:end]

    records = []
    for doc in batch_docs:
        records.append({
            "id": str(uuid4()),
            "text": doc.page_content,
        })

    index.upsert_records(
        namespace="wine-review2-ns",
        records=records
    )

    print(
        f"배치 {(start // BATCH_SIZE) + 1} 완료 "
        f"({len(batch_docs)}개 / 누적 {min(end, len(chunks))}개)"
    )

print("✅ 모든 문서 업로드 완료")


배치 1 완료 (50개 / 누적 50개)
배치 2 완료 (50개 / 누적 100개)
배치 3 완료 (50개 / 누적 150개)
배치 4 완료 (50개 / 누적 200개)
배치 5 완료 (50개 / 누적 250개)
배치 6 완료 (50개 / 누적 300개)
배치 7 완료 (50개 / 누적 350개)
배치 8 완료 (50개 / 누적 400개)
배치 9 완료 (50개 / 누적 450개)
배치 10 완료 (50개 / 누적 500개)
배치 11 완료 (50개 / 누적 550개)
배치 12 완료 (50개 / 누적 600개)
배치 13 완료 (50개 / 누적 650개)
배치 14 완료 (50개 / 누적 700개)
배치 15 완료 (50개 / 누적 750개)
배치 16 완료 (50개 / 누적 800개)
배치 17 완료 (50개 / 누적 850개)
배치 18 완료 (50개 / 누적 900개)
배치 19 완료 (50개 / 누적 950개)
배치 20 완료 (50개 / 누적 1000개)
배치 21 완료 (50개 / 누적 1050개)
배치 22 완료 (50개 / 누적 1100개)
배치 23 완료 (50개 / 누적 1150개)
배치 24 완료 (50개 / 누적 1200개)
배치 25 완료 (50개 / 누적 1250개)
배치 26 완료 (50개 / 누적 1300개)
배치 27 완료 (50개 / 누적 1350개)
배치 28 완료 (50개 / 누적 1400개)
배치 29 완료 (50개 / 누적 1450개)
배치 30 완료 (50개 / 누적 1500개)
배치 31 완료 (50개 / 누적 1550개)
배치 32 완료 (50개 / 누적 1600개)
배치 33 완료 (50개 / 누적 1650개)
배치 34 완료 (50개 / 누적 1700개)
배치 35 완료 (50개 / 누적 1750개)
배치 36 완료 (50개 / 누적 1800개)
배치 37 완료 (50개 / 누적 1850개)
배치 38 완료 (50개 / 누적 1900개)
배치 39 완료 (50개 / 누적 1950개)
배치 40 완료 (50개 /

PineconeApiException: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 05 Jan 2026 05:29:10 GMT', 'Content-Type': 'text/plain; charset=utf-8', 'Content-Length': '332', 'Connection': 'keep-alive', 'x-pinecone-api-version': '2025-04', 'x-envoy-upstream-service-time': '1788', 'x-pinecone-response-duration-ms': '1789', 'server': 'envoy'})
HTTP response body: {"error":{"code":"RESOURCE_EXHAUSTED","message":"{\"error\":{\"code\":\"RESOURCE_EXHAUSTED\",\"message\":\"Request failed. You've reached the max tokens per minute (250000) for model 'llama-text-embed-v2' and input_type 'passage' for the current project. To increase this limit, upgrade your plan.\"},\"status\":429}"},"status":429}


# 로컬모델로 임베딩 후 파인콘 DB에 밀어 넣기

In [36]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

pc.create_index(
    name="wine-review-v2",
    host="https://wine-review-v2-ndidgu3.svc.aped-4627-b74a.pinecone.io"
    dimension=384,          # 🔥 핵심
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (319617691.py, line 8)

In [31]:
from sentence_transformers import SentenceTransformer

# 가볍고 빠르고 RAG 표준
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Admin\miniconda3\envs\lc_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [32]:
texts = [doc.page_content for doc in chunks]

embeddings = model.encode(
    texts,
    batch_size=64,
    show_progress_bar=True,
    normalize_embeddings=True
)


Batches:   0%|          | 0/2031 [00:00<?, ?it/s]

In [37]:
index = pc.Index(
    name="wine-review-v2",
    host="https://wine-review-v2-ndidgu3.svc.aped-4627-b74a.pinecone.io"
)

In [38]:
from uuid import uuid4

BATCH_SIZE = 200

for start in range(0, len(chunks), BATCH_SIZE):
    end = start + BATCH_SIZE

    vectors = []
    for i in range(start, min(end, len(chunks))):
        vectors.append({
            "id": str(uuid4()),
            "values": embeddings[i].tolist(),
            "metadata": {
                "source": chunks[i].metadata.get("source", ""),
                "row": chunks[i].metadata.get("row", -1)
            }
        })

    index.upsert(
        vectors=vectors,
        namespace="wine-review-v2"
    )

    print(f"업로드 {min(end, len(chunks))}/{len(chunks)} 완료")


업로드 200/129982 완료
업로드 400/129982 완료
업로드 600/129982 완료
업로드 800/129982 완료
업로드 1000/129982 완료
업로드 1200/129982 완료
업로드 1400/129982 완료
업로드 1600/129982 완료
업로드 1800/129982 완료
업로드 2000/129982 완료
업로드 2200/129982 완료
업로드 2400/129982 완료
업로드 2600/129982 완료
업로드 2800/129982 완료
업로드 3000/129982 완료
업로드 3200/129982 완료
업로드 3400/129982 완료
업로드 3600/129982 완료
업로드 3800/129982 완료
업로드 4000/129982 완료
업로드 4200/129982 완료
업로드 4400/129982 완료
업로드 4600/129982 완료
업로드 4800/129982 완료
업로드 5000/129982 완료
업로드 5200/129982 완료
업로드 5400/129982 완료
업로드 5600/129982 완료
업로드 5800/129982 완료
업로드 6000/129982 완료
업로드 6200/129982 완료
업로드 6400/129982 완료
업로드 6600/129982 완료
업로드 6800/129982 완료
업로드 7000/129982 완료
업로드 7200/129982 완료
업로드 7400/129982 완료
업로드 7600/129982 완료
업로드 7800/129982 완료
업로드 8000/129982 완료
업로드 8200/129982 완료
업로드 8400/129982 완료
업로드 8600/129982 완료
업로드 8800/129982 완료
업로드 9000/129982 완료
업로드 9200/129982 완료
업로드 9400/129982 완료
업로드 9600/129982 완료
업로드 9800/129982 완료
업로드 10000/129982 완료
업로드 10200/129982 완료
업로드 10400/129982 완료
업로드 10600/129